In [1]:
# Imports

import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

import psycopg2
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from scipy.stats import pearsonr
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import SGDRegressor
import statsmodels.api as sm
from sklearn.model_selection import GridSearchCV


pd.set_option('display.max_rows', None) #magic command
pd.set_option('display.max_columns', None)



In [2]:
conn = psycopg2.connect("dbname=mimic user=mimic password=Mimic@4@plhi")
cur = conn.cursor()

In [3]:
# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "Machine_learning_ch-3"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [4]:
def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

df_labevents_1 = create_pandas_table("""select itemid, label, loinc_code from mimic_hosp.d_labitems """)
print (df_labevents_1.shape)
df_labevents_1.head(1625)
# df_labevents_1['loinc_code'].isnull().sum

(1625, 3)


,itemid,label,loinc_code
0,51898,,None
1,51532,11-Deoxycorticosterone,None
2,51952,17-Hydroxycorticosteroids,None
3,51953,"17-Ketosteroids, Urine",None
4,52063,24 Hr,None
5,51066,24 hr Calcium,None
6,51067,24 hr Creatinine,None
7,51068,24 hr Protein,None
8,50853,25-OH Vitamin D,None
9,51533,3t,None


In [5]:
# dropping the Null values
df_labevents_1.dropna(inplace = True)
df_labevents_1

,itemid,label,loinc_code
12,51535,5' Nucleotidase,1690-7
17,52064,Absolute Basophil Count,704-7
20,51130,Absolute CD3 Count,8124-0
21,51131,Absolute CD4 Count,8128-1
23,51132,Absolute CD8 Count,8138-0
24,52068,Absolute Eosinophil Count,711-2
27,51133,Absolute Lymphocyte Count,731-0
29,52069,Absolute Monocyte Count,742-7
31,52070,Absolute Neutrophil Count,751-8
34,51134,Acanthocytes,7789-1


In [6]:
df_labevents = create_pandas_table("""select subject_id, hadm_id, itemid, flag, value from mimic_hosp.labevents""")
print (df_labevents.shape)
df_labevents.head(50) 

(122289828, 5)


,subject_id,hadm_id,itemid,flag,value
0,15449134,27159811.0,51006,None,18
1,15449134,27159811.0,51146,None,0.6
2,15449134,27159811.0,51200,None,3.6
3,15449134,27159811.0,51221,abnormal,30.9
4,15449134,27159811.0,51222,abnormal,10.4
5,15449134,27159811.0,51244,None,24.4
6,15449134,27159811.0,51248,None,28.6
7,15449134,27159811.0,51249,None,33.6
8,15449134,27159811.0,51250,None,85
9,15449134,27159811.0,51254,None,5.8


In [7]:
itemid_label = df_labevents_1.set_index('itemid').to_dict()['label']
itemid_label

{51535: "5' Nucleotidase",
 52064: 'Absolute Basophil Count',
 51130: 'Absolute CD3 Count',
 51131: 'Absolute CD4 Count',
 51132: 'Absolute CD8 Count',
 52068: 'Absolute Eosinophil Count',
 51133: 'Absolute Lymphocyte Count',
 52069: 'Absolute Monocyte Count',
 52070: 'Absolute Neutrophil Count',
 51134: 'Acanthocytes',
 50856: 'Acetaminophen',
 50857: 'Acetone',
 50858: 'Acid Phosphatase',
 51539: 'Acid Phosphatase, Prostatic Fraction',
 51540: 'Adrenocorticotrophic Hormone',
 50861: 'Alanine Aminotransferase (ALT)',
 51542: '(Albumin)',
 50862: 'Albumin',
 51070: 'Albumin/Creatinine, Urine',
 51544: 'Aldosterone',
 50863: 'Alkaline Phosphatase',
 51545: 'Alpha-1',
 51547: 'Alpha-2',
 50864: 'Alpha-Fetoprotein',
 51462: 'Amorphous Crystals',
 50867: 'Amylase',
 50868: 'Anion Gap',
 51137: 'Anisocytosis',
 51556: 'Anti-Microsomal Antibodies',
 51557: 'Anti-Microsomal Antibody',
 50873: 'Anti-Nuclear Antibody',
 50874: 'Anti-Nuclear Antibody, Titer',
 50877: 'Anti-Thyroglobulin Antibodi

In [8]:
df_labevents['label'] = df_labevents['itemid'].map(itemid_label)

In [9]:
df_labevents.head(50)

,subject_id,hadm_id,itemid,flag,value,label
0,15449134,27159811.0,51006,None,18,Urea Nitrogen
1,15449134,27159811.0,51146,None,0.6,Basophils
2,15449134,27159811.0,51200,None,3.6,Eosinophils
3,15449134,27159811.0,51221,abnormal,30.9,Hematocrit
4,15449134,27159811.0,51222,abnormal,10.4,Hemoglobin
5,15449134,27159811.0,51244,None,24.4,Lymphocytes
6,15449134,27159811.0,51248,None,28.6,MCH
7,15449134,27159811.0,51249,None,33.6,MCHC
8,15449134,27159811.0,51250,None,85,MCV
9,15449134,27159811.0,51254,None,5.8,Monocytes


In [10]:
df_labevents_ = df_labevents.copy()
df_labevents_.shape

(122289828, 6)

In [11]:
df_labevents_ = df_labevents_.loc[df_labevents_['label'].isin(['Absolute Lymphocyte Count', 'Absolute Neutrophil Count', 'WBC',  'RBC', 'RDW', 'Basophils', 'Eosinophils','Neutrophils', 'Monocytes',
                                                                  'Hematocrit', 'Hemoglobin', 'MCV', 'Platelet Count', 'Alanine Aminotransferase (ALT)', 'Asparate Aminotransferase (AST)', 'Alkaline Phosphatase','PT', 'Albumin', 'Globulin',
                                                                 'Bilirubin, Total', 'Potassium', 'Sodium', 'Creatine Kinase (CK)', 'Cholesterol, HDL', 'Cholesterol, LDL, Calculated', 'Cholesterol, Total', 'C-Reactive Protein', 
                                                                   'Creatinine', 'Urea Nitrogen', 'Uric Acid', 'Lactate Dehydrogenase (LD)',
                                                    ])]

In [12]:
df_labevents_.head()

,subject_id,hadm_id,itemid,flag,value,label
0,15449134,27159811.0,51006,None,18,Urea Nitrogen
1,15449134,27159811.0,51146,None,0.6,Basophils
2,15449134,27159811.0,51200,None,3.6,Eosinophils
3,15449134,27159811.0,51221,abnormal,30.9,Hematocrit
4,15449134,27159811.0,51222,abnormal,10.4,Hemoglobin


In [15]:
# Replacing none as normal

In [16]:
df_labevents_['flag'] = df_labevents_['flag'].fillna('normal')

In [17]:
df_labevents_.dropna(inplace = True)

In [18]:
df_labevents_["lab_features"] = df_labevents_["flag"] + df_labevents_["label"]

In [19]:
df_labevents_.head(10)

,subject_id,hadm_id,itemid,flag,value,label,lab_features
0,15449134,27159811.0,51006,normal,18,Urea Nitrogen,normalUrea Nitrogen
1,15449134,27159811.0,51146,normal,0.6,Basophils,normalBasophils
2,15449134,27159811.0,51200,normal,3.6,Eosinophils,normalEosinophils
3,15449134,27159811.0,51221,abnormal,30.9,Hematocrit,abnormalHematocrit
4,15449134,27159811.0,51222,abnormal,10.4,Hemoglobin,abnormalHemoglobin
8,15449134,27159811.0,51250,normal,85,MCV,normalMCV
9,15449134,27159811.0,51254,normal,5.8,Monocytes,normalMonocytes
10,15449134,27159811.0,51256,normal,65.7,Neutrophils,normalNeutrophils
11,15449134,27159811.0,51265,normal,226,Platelet Count,normalPlatelet Count
12,15449134,27159811.0,51277,normal,15.5,RDW,normalRDW


# for CBC's

In [ ]:
# df_labevents_bld_val = df_labevents_.loc[df_labevents_['label'].isin([ 'Absolute Lymphocyte Count', 'Absolute Neutrophil Count', 'WBC',  'RBC', 'RDW', 'Basophils', 'Eosinophils','Neutrophils', 'Monocytes',
#                                                                   'Hematocrit', 'Hemoglobin', 'MCV', 'Platelet Count'
#                                                                  ])]

In [ ]:
# df_labevents_bld_val['lab_features'].value_counts()

In [ ]:
#verify none in flag
# df_labevents_bld_val['flag'].isnull().sum() 

In [ ]:
# df_labevents_bld_val['flag'].value_counts()

In [ ]:
# df_labevents_bld_val['lab_features'].isnull().sum()

In [ ]:
# df_labevents_bld_val.head(10)


In [ ]:
# df_labevents_bld_val.dropna(inplace = True)

In [ ]:
# df_labevents_bld_val['cat'] = df_labevents_bld_val['lab_features']
# lab_list = df_labevents_bld_val.groupby('hadm_id')['cat'].apply(list).reset_index()
# lab_list.head(50)

In [ ]:

# lab_item = pd.get_dummies(lab_list['cat'].apply(pd.Series).stack()).sum(level=0)

# lab_item.head(10)

In [ ]:
# lab_item = lab_item.join(lab_list['hadm_id'], how="outer")
# lab_item.head()

In [ ]:
# lab_item.to_csv('lab_item.csv', index = False)

#  for lfts

In [ ]:
# df_labevents_lfts = df_labevents_.loc[df_labevents_['label'].isin(['Alanine Aminotransferase (ALT)', 'Asparate Aminotransferase (AST)', 'Alkaline Phosphatase','PT', 'Albumin', 'Globulin',
#                                                                  'Bilirubin, Total', 'Potassium', 'Sodium', 'Creatine Kinase (CK)',])]

In [ ]:
# df_labevents_lfts['lab_features'].value_counts()

In [ ]:
# df_labevents_lfts.dropna(inplace = True)

In [ ]:
# df_labevents_lfts['cat'] = df_labevents_lfts['lab_features']
# lab_list1 = df_labevents_lfts.groupby('hadm_id')['cat'].apply(list).reset_index()
# lab_list1.head(50)

In [ ]:

# lab_item1 = pd.get_dummies(lab_list1['cat'].apply(pd.Series).stack()).sum(level=0)

# lab_item1.head(10)

In [ ]:
# lab_item1 = lab_item1.join(lab_list1['hadm_id'], how="outer")
# lab_item1.head()

In [ ]:
# lab_item1.to_csv('lab_item1.csv', index = False)

# chols and RFTs

In [ ]:
# df_labevents_rfts = df_labevents_.loc[df_labevents_['label'].isin([ 'Cholesterol, HDL', 'Cholesterol, LDL, Calculated', 'Cholesterol, Total', 'C-Reactive Protein', 
#                                                                    'Creatinine', 'Urea Nitrogen', 'Uric Acid', 'Lactate Dehydrogenase (LD)'
#                                                     ])]

In [ ]:
# df_labevents_rfts['lab_features'].value_counts()

In [ ]:
# df_labevents_rfts.dropna(inplace = True)

In [ ]:
# df_labevents_rfts['cat'] = df_labevents_rfts['lab_features']
# lab_list2 = df_labevents_rfts.groupby('hadm_id')['cat'].apply(list).reset_index()
# lab_list2.head(50)

In [ ]:
# lab_item2 = pd.get_dummies(lab_list2['cat'].apply(pd.Series).stack()).sum(level=0)

# lab_item2.head(10)

In [ ]:
# lab_item2 = lab_item2.join(lab_list2['hadm_id'], how="outer")
# lab_item2.head()

In [ ]:
# lab_item2.to_csv('lab_item2.csv', index = False)

In [ ]:
x = pd.read_csv('lab_item.csv') 
y = pd.read_csv('lab_item1.csv') 
z = pd.read_csv('lab_item2.csv') 

In [ ]:
x.isnull().sum()

In [ ]:
y.isnull().sum()

In [ ]:
z.isnull().sum()

In [ ]:
# compile the list of dataframes you want to merge
from functools import reduce

data_frames = [x, y, z]
lab_data = reduce(lambda  left,right: pd.merge(left,right,on=['hadm_id'],
                                            how='outer'), data_frames)


In [ ]:
lab_data.head()

In [ ]:
lab_data.info()

In [ ]:
lab_data.to_csv('lab_data.csv', index = False)

In [ ]:
lab_data.isnull().sum()

In [ ]:
lab_data.shape